# Génération de partition de musique : réseau LSTM

**/!\ Les notes et leurs durées sont liées lors de la phase d'apprentissage**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer

from typing import Dict, List, Optional, Sequence, Tuple
import numpy as np
import pandas as pd
import pretty_midi
import music21

import json
import os
import sys
sys.path.append('C:/Users/melan/Documents/M2_S10 IAFA/CHEF D\'OEUVRE')
import extract_data as ed
import visualization as vz

c:\users\melan\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


### Choix de la partie à générer

In [2]:
part = 'A'

### Préparation du dataset

In [3]:
# Creating the required variables
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
with open("data/data.json",'r') as file:
    data = json.load(file)
                
filenames = [data[i]['title'] for i in range(len(data))]
print('Number of scores:', len(filenames))

Number of scores: 171


In [5]:
# Enregistrement des listes des parties A, B et C de toutes les melodies de data.json
all_datasets = ed.json_into_part_melody("data/data.json")
print("Number of scores for :")
print("  - part A :",all_datasets[0])
print("  - part B :",all_datasets[1])
print("  - part C :",all_datasets[2])

Number of scores for :
  - part A : 169
  - part B : 166
  - part C : 98


In [6]:
with open("data/dataset"+part+".json",'r') as file:
    dataset = json.load(file)

In [7]:
from music21 import note

# Créez un objet Note avec la note E6
e6_note = note.Note("E6")

# Obtenez le numéro MIDI de la note
midi_number = e6_note.pitch.midi

# Affichez le résultat
print(f"La note E6 a un numéro MIDI de {midi_number}")


La note E6 a un numéro MIDI de 88


Choix de l'encodage
- note : convertie avec music21
- temps : duration * 0.1

=> note finale : note + temps

In [8]:
def tokenize_and_encode_notes(score):
    
    notes = [n for n in score.split(",")]
    encoded_notes = []

    for n in notes:
        
        splitted = n.replace(" ","").split('-')

        if len(splitted) == 2:
            
            if splitted[0] == 'rest':
                final_encoding = float(splitted[1]) * 0.1
                encoded_notes.append(final_encoding)
                
            else:
                only_note,duration = note.Note(splitted[0]).pitch.midi,float(splitted[1])
                final_encoding = only_note + duration * 0.1
                encoded_notes.append(final_encoding)
        
        else:
            only_note = note.Note("b".join(substring for substring in splitted[:-1])).pitch.midi
            duration = float(splitted[-1])
            final_encoding = only_note + duration * 0.1
            encoded_notes.append(final_encoding)
            
    return encoded_notes

### Création du dataset d'entraînement

In [40]:
all_notes = []

for i in range(len(dataset)):
    
    notes = pd.DataFrame(tokenize_and_encode_notes(dataset[i]))
    all_notes.append(notes)

all_notes = pd.concat(all_notes)

n_notes = len(all_notes)
print('Number of notes parsed for part',part,':', n_notes)

train_notes = np.array(all_notes)

notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)

notes_ds.element_spec

Number of notes parsed for part A : 19175


TensorSpec(shape=(1,), dtype=tf.float64, name=None)

In [23]:
def create_sequences(dataset, seq_length):
    """Returns TF Dataset of sequence and label examples."""
    
    seq_length = seq_length+1

    # Take 1 extra for the labels
    windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)

    # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

    # Split the labels
    def split_labels(sequences):
        inputs = sequences[:-1]
        labels_dense = sequences[-1]

        return inputs,labels_dense

    return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    
seq_length = 50
seq_ds = create_sequences(notes_ds, seq_length)

In [24]:
batch_size = 32
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))
print(train_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 50, 1), dtype=tf.float64, name=None), TensorSpec(shape=(32, 1), dtype=tf.float64, name=None))>


### Développement du modèle

In [25]:
def mse_with_positive_pressure(y_true, y_pred):
    mse = (y_true - y_pred) ** 2
    positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
    return tf.reduce_mean(mse + positive_pressure)

In [26]:
# Developing the model

input_shape = (seq_length,1)
learning_rate = 0.005

inputs = Input(input_shape)
x = LSTM(64)(inputs)

outputs = {'note': Dense(128, name='note')(x),
          }

model = Model(inputs, outputs)

loss = {'note': SparseCategoricalCrossentropy(from_logits=True),
       }

optimizer = Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=optimizer)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 1)]           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 note (Dense)                (None, 128)               8320      
                                                                 
Total params: 25216 (98.50 KB)
Trainable params: 25216 (98.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Creating the necessary callbacks

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./training_checkpoints/ckpt_{epoch}', save_weights_only=True),
             tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, 
                                              verbose=1, restore_best_weights=True),]

In [28]:
# Compiling and fitting the model

model.compile(loss = loss, 
              optimizer = optimizer)

epochs = 50

history = model.fit(train_ds, 
                    epochs=epochs,
                   callbacks=callbacks,)

Epoch 1/50
597/597 [==============================] - 16s 18ms/step - loss: 3.1017
Epoch 2/50
597/597 [==============================] - 11s 19ms/step - loss: 2.8639
Epoch 3/50
597/597 [==============================] - 11s 19ms/step - loss: 2.7798
Epoch 4/50
597/597 [==============================] - 13s 22ms/step - loss: 2.7441
Epoch 5/50
597/597 [==============================] - 13s 22ms/step - loss: 2.7362
Epoch 6/50
597/597 [==============================] - 13s 21ms/step - loss: 2.7168
Epoch 7/50
597/597 [==============================] - 12s 20ms/step - loss: 2.6983
Epoch 8/50
597/597 [==============================] - 12s 21ms/step - loss: 2.6889
Epoch 9/50
597/597 [==============================] - 12s 20ms/step - loss: 2.6824
Epoch 10/50
597/597 [==============================] - 12s 20ms/step - loss: 2.6753
Epoch 11/50
597/597 [==============================] - 12s 20ms/step - loss: 2.6890
Epoch 12/50
597/597 [==============================] - 12s 20ms/step - loss: 2.6700
E

### Génération de notes

In [29]:
def predict_next_note(notes, model, temperature = 1.0):
    """Generates a note IDs using a trained sequence model."""

    assert temperature > 0

    # Add batch dimension
    inputs = tf.expand_dims(notes, 0)

    predictions = model.predict(inputs)
    note_logits = predictions['note']

    note_logits /= temperature
    note = tf.random.categorical(note_logits, num_samples=1)
    note = tf.squeeze(note, axis=-1)

    return float(note)

In [30]:
def decoding_note(generated_note):
    
    quarter_duration = [0.0625,0.125,0.25,0.5,1.0,2.0]
    
    if int(generated_note) == 0:
        duration = min(quarter_duration, key=lambda x:abs(x-(10 * generated_note)))
        return "rest-"+str(duration)
    
    else:
        duration = min(quarter_duration, key=lambda x:abs(x-(10 * (generated_note - round(generated_note)))))
        note = music21.pitch.Pitch()
        note.midi = round(generated_note)
        return str(note.nameWithOctave)+"-"+str(duration)

In [53]:
def generated_notes_to_dict(score,temperature=2.0,num_predictions=8):
    
    raw_notes = tokenize_and_encode_notes(score)
    sample_notes = np.array(raw_notes)
    input_notes = sample_notes[:8]
    
    original_notes = [decoding_note(n) for n in sample_notes]
    initial_notes = original_notes[:8]
    
    generated_notes = []
    
    for _ in range(num_predictions):
        
        note = predict_next_note(input_notes, model)
        dec_note = decoding_note(note)
        generated_notes.append(dec_note)
        
        input_notes = np.delete(note, 0)
        input_notes = np.append(note, np.expand_dims(note, 0))
    
    return {'Title':'','Part':part,'Key':"F major",'Start_sequence':initial_notes,'Generated':generated_notes,'Original':original_notes}

In [54]:
generated_scores = []

for f in dataset[:10]:
    
    gen_notes = generated_notes_to_dict(f)
    generated_scores.append(gen_notes)
    
with open('Generated/LSTM_'+part+'.json','w') as file:
    json.dump(generated_scores,file) 

1/1 [==============================] - 0s 18ms/step
B-5-0.0625
1/1 [==============================] - 0s 16ms/step
E7-0.0625
1/1 [==============================] - 0s 31ms/step
E-6-0.0625
1/1 [==============================] - 0s 16ms/step
E6-0.0625
1/1 [==============================] - 0s 16ms/step
E-6-0.0625
1/1 [==============================] - 0s 31ms/step
E-6-0.0625
1/1 [==============================] - 0s 32ms/step
F#7-0.0625
1/1 [==============================] - 0s 31ms/step
D6-0.0625
1/1 [==============================] - 0s 16ms/step
B4-0.0625
1/1 [==============================] - 0s 31ms/step
C6-0.0625
1/1 [==============================] - 0s 32ms/step
B-5-0.0625
1/1 [==============================] - 0s 16ms/step
E7-0.0625
1/1 [==============================] - 0s 31ms/step
G7-0.0625
1/1 [==============================] - 0s 16ms/step
A5-0.0625
1/1 [==============================] - 0s 31ms/step
D6-0.0625
1/1 [==============================] - 0s 38ms/step
E-6-0.0625
1

In [36]:
music21.environment.set('musescoreDirectPNGPath',str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
music21.environment.set('musicxmlPath', str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
vz.show_all_generated("Generated/LSTM_"+part+".json").show("musicxml")